# Minimal Example using YOLOv11-Tracking
## Install libraries 
Run this once in the beginning or to get the updated code for the course

In [1]:
!pip install opencv-python
!pip install ffmpegcv
!pip install scipy
!pip install numpy
!pip install tqdm
!pip install scikit-image
!pip install pillow
!pip install ffmpeg
!pip install --upgrade --force-reinstall git+https://github.com/leon-etienne/gensurv.git@main

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/leon-etienne/gensurv.git (to revision main) to c:\users\etien\appdata\local\temp\pip-req-build-6hckpo42
  Resolved https://github.com/leon-etienne/gensurv.git to commit d5f66001218ef714a0e70608704580a5edb3b722
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  

  Running command git clone --filter=blob:none --quiet https://github.com/leon-etienne/gensurv.git 'C:\Users\etien\AppData\Local\Temp\pip-req-build-6hckpo42'


In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from gensurv import *

## Load the video
Takes all frames from the video from second 0 to second 5 and stores them in a list of frames `video_frames`.
Also stores the `fps` and `duration` as numbers. The number of frames is just the length of the `video_frames` list.

In [4]:
video_frames, fps, duration = get_video_frames("Town.mp4", start=0, end=5)
number_of_frames = len(video_frames)

Number of frames: 125
Frames per second (fps): 25.00333288894814
Duration: 300.04 seconds


## Load the YOLO model

This loads the yolov11 model specialized in segmentation. 
There are other tasks YOLO can do, you can read more about it in the official [ultralytics documentation](https://docs.ultralytics.com/tasks/).

In [80]:
# Prepare Model
model = YOLO("yolo11n-seg.pt")  # Load an official Segment model
print(model.names)

FileNotFoundError: [Errno 2] No such file or directory: 'yolo11n-seg.pt'

## Run the tracking

This code goes through all frames of the video and tracks objects of all classes of the model.
It starts a tracker that can be used to show traces of objects over a number of frames.
You can 

In [81]:
def process_video_frames(video_frames):
    previous_frame = np.zeros_like(video_frames[0])
    
    processed_frames = []

    ### Reset Ids so they stay consistent###
    if model.predictor is not None:
        model.predictor.trackers[0].reset_id()

    ### Start Tracker ###
    process_results_to_tacks = start_results_to_tracks()
    
    for index, current_frame in enumerate(tqdm(video_frames)):
        processed_frame = current_frame.copy()

        # Track the material
        results = model.track(processed_frame, persist=True, verbose=False)

        ### A) Calculations including the previous frame ####

        person_masks = process_results_to_masks_normalized(results, processed_frame)

        processed_frame = processed_frame * person_masks

        #####################################################
        
        previous_frame = processed_frame

        ### B) After Calculations with the previous frame ###

        # Generate a frame containing the tracks over 50 frames
        tracking_frame = process_results_to_tacks(results, processed_frame, max_tracks=50, color=(255, 0, 255), thickness=3)

        # Overdraw the tracks
        processed_frame = combine_images_with_mask(processed_frame, tracking_frame)

        ####################################################
        processed_frames.append(processed_frame)

    return processed_frames

processed_frames = process_video_frames(video_frames)

save_video_frames(processed_frames, "yolo_tracking.mp4", fps, display_video=False)
create_video_browser("/home/jovyan")

100%|██████████| 125/125 [00:17<00:00,  7.03it/s]


Video saved to: 20241127_134256_yolo_tracking.mp4
